In [1]:
%pylab inline
import torch
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch import nn, optim
import VAE

Populating the interactive namespace from numpy and matplotlib


In [2]:
# declare loss functions

def log_bernoulli_loss(x_hat, x):
    
    loss = - torch.sum(x * torch.log(x_hat) + (1-x) *(torch.log(1-x_hat)), 1)
    
    return torch.sum(loss)


def KL_loss(mu, logvar):
    
    D = torch.FloatTensor([mu.size(1)])
    log_D = torch.log(D)
    sum_logvar = torch.sum(logvar, 1)
    norm_var = torch.sum(torch.exp(logvar), 1)
    norm_mu = torch.sum(mu * mu, 1)
    loss = (log_D - sum_logvar + norm_var + norm_mu - D)/2
    
    return torch.sum(loss)


def loss_function(x_hat, x, mu, logvar):
    
    log_bernoulli = log_bernoulli_loss(x_hat, x)
    KL = KL_loss(mu, logvar)
    
    return log_bernoulli+KL

In [3]:
from torch import nn
from torch.nn import functional as F 

class VAE(nn.Module):
    def __init__(self, fc1_dims, fc21_dims, fc22_dims, fc3_dims, fc4_dims):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(*fc1_dims)
        self.fc21 = nn.Linear(*fc21_dims)
        self.fc22 = nn.Linear(*fc22_dims)
        self.fc3 = nn.Linear(*fc3_dims)
        self.fc4 = nn.Linear(*fc4_dims)

    def encode(self, x):
        embedding = F.relu(self.fc1(x))
        mu = F.sigmoid(self.fc21(embedding))
        logvar = F.tanh(self.fc22(embedding))
        return mu, logvar

    def reparameterize(self, mu, logvar):
        epsilon = torch.normal(torch.zeros(logvar.size()), torch.ones(logvar.size()))
        sigma = torch.sqrt(torch.exp(logvar))
        z = mu + epsilon * logvar
        return z

    def decode(self, z):
        x_hat =  F.sigmoid(self.fc4(self.fc3(z)))
        return x_hat

    def forward(self, x):
        x = x.view(-1, 784)
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        return x_hat, mu, logvar

In [4]:
def train(epoch, train_loader, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data.view(-1, 784), mu, logvar)
        loss.backward()
        train_loss += loss.data
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [5]:
# Hyperparameters
fc1_dims = (784,400)
fc21_dims = (400,20)
fc22_dims = (400,20)
fc3_dims = (20, 400)
fc4_dims = (400,784)
lr = 0.001
batch_size = 32
epochs = 10

In [6]:
# Load data
train_data = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size, shuffle=True, **{})

# Init model
VAE_MNIST = VAE(fc1_dims=fc1_dims, fc21_dims=fc21_dims, fc22_dims=fc22_dims, fc3_dims=fc3_dims, fc4_dims=fc4_dims)

# Init optimizer
optimizer = optim.Adam(VAE_MNIST.parameters(), lr=lr)

# Train
for epoch in range(1, epochs + 1):
    train(epoch, train_loader, VAE_MNIST, optimizer)

Train Epoch: 1 [0/60000 (0%)]	Loss: 550.830078
Train Epoch: 1 [3200/60000 (5%)]	Loss: 211.150452
Train Epoch: 1 [6400/60000 (11%)]	Loss: 170.457962
Train Epoch: 1 [9600/60000 (16%)]	Loss: 157.447418
Train Epoch: 1 [12800/60000 (21%)]	Loss: 167.667160
Train Epoch: 1 [16000/60000 (27%)]	Loss: 147.450348
Train Epoch: 1 [19200/60000 (32%)]	Loss: 136.125763
Train Epoch: 1 [22400/60000 (37%)]	Loss: 128.148544
Train Epoch: 1 [25600/60000 (43%)]	Loss: 142.303574
Train Epoch: 1 [28800/60000 (48%)]	Loss: 129.733139
Train Epoch: 1 [32000/60000 (53%)]	Loss: 121.319611
Train Epoch: 1 [35200/60000 (59%)]	Loss: 116.309731
Train Epoch: 1 [38400/60000 (64%)]	Loss: 129.097626
Train Epoch: 1 [41600/60000 (69%)]	Loss: 107.516060
Train Epoch: 1 [44800/60000 (75%)]	Loss: 113.921371
Train Epoch: 1 [48000/60000 (80%)]	Loss: 110.488884
Train Epoch: 1 [51200/60000 (85%)]	Loss: 115.985474
Train Epoch: 1 [54400/60000 (91%)]	Loss: 109.106956
Train Epoch: 1 [57600/60000 (96%)]	Loss: 113.194550
====> Epoch: 1 Averag

Train Epoch: 9 [12800/60000 (21%)]	Loss: 82.419060
Train Epoch: 9 [16000/60000 (27%)]	Loss: 74.994934
Train Epoch: 9 [19200/60000 (32%)]	Loss: 86.295601
Train Epoch: 9 [22400/60000 (37%)]	Loss: 82.733406
Train Epoch: 9 [25600/60000 (43%)]	Loss: 92.272247
Train Epoch: 9 [28800/60000 (48%)]	Loss: 86.362236
Train Epoch: 9 [32000/60000 (53%)]	Loss: 86.960068
Train Epoch: 9 [35200/60000 (59%)]	Loss: 78.461273
Train Epoch: 9 [38400/60000 (64%)]	Loss: 88.502586
Train Epoch: 9 [41600/60000 (69%)]	Loss: 85.809181
Train Epoch: 9 [44800/60000 (75%)]	Loss: 75.637283
Train Epoch: 9 [48000/60000 (80%)]	Loss: 88.663155
Train Epoch: 9 [51200/60000 (85%)]	Loss: 86.481720
Train Epoch: 9 [54400/60000 (91%)]	Loss: 82.719078
Train Epoch: 9 [57600/60000 (96%)]	Loss: 88.493736
====> Epoch: 9 Average loss: 84.8151
Train Epoch: 10 [0/60000 (0%)]	Loss: 92.469582
Train Epoch: 10 [3200/60000 (5%)]	Loss: 86.385712
Train Epoch: 10 [6400/60000 (11%)]	Loss: 88.447319
Train Epoch: 10 [9600/60000 (16%)]	Loss: 80.475220

In [ ]:
### Let's check if the reconstructions make sense
# Set model to test mode
VAE_MNIST.eval()
    
# Reconstructed
train_data_plot = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor())

train_loader_plot = torch.utils.data.DataLoader(train_data_plot,
                                           batch_size=1, shuffle=False, **{})

for batch_idx, (data, _) in enumerate(train_loader_plot):
    x_hat, mu, logvar = VAE_MNIST(data)
    plt.imshow(x_hat.view(1,28,28).squeeze().data.numpy(), cmap='gray')
    plt.title('%i' % train_data.train_labels[batch_idx])
    plt.show()
    if batch_idx == 3:
        break